change the action
and visualize the grasping point

In [ ]:
import os
import cv2
import open3d as o3d

# Define directories, camera intrinsics, and new directories for resized images
directories = {
    # "front": {
    #     "rgb": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_images/rgb",
    #     "depth": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_images/depth",
    #     "point": "/media/ubb/My Passport/try/lsshirt_bluedot_2/pointcloud",
    #     "intrinsics": (1070.1000, 1070.4301, 939.8100, 549.6520)}
    #      ,
    "top": {
        "rgb": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_top_images/rgb",
        "depth": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_top_images/depth",
        "point": "/media/ubb/My Passport/try/lsshirt_bluedot_2/pointcloud_top",
        "intrinsics": (1060.4000, 1060.0100, 946.0600, 496.6630)},
    "side": {
        "rgb": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_images/rgb",
        "depth": "/media/ubb/My Passport/try/lsshirt_bluedot_2/bag_side_images/depth",
        "point": "/media/ubb/My Passport/try/lsshirt_bluedot_2/pointcloud_side",
        "intrinsics": (1066.4000, 1065.9900, 970.8000, 547.1950)}   
}

def process_images_and_generate_pointclouds(value):
    rgb_dir = value["rgb"]
    depth_dir = value["depth"]
    point_dir = value["point"]
    intrinsics = value["intrinsics"]

    if not os.path.exists(point_dir):
        os.makedirs(point_dir)


    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_files = sorted([f for f in os.listdir(depth_dir) if f.endswith('.png')])
    pointcloud_count = 0

    for rgb_file, depth_file in zip(rgb_files, depth_files):
        rgb_image_path = os.path.join(rgb_dir, rgb_file)
        depth_image_path = os.path.join(depth_dir, depth_file)
        
        color_raw = cv2.imread(rgb_image_path)
        color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
        depth_raw = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)
        
        # print(f"Original RGB range: {color_raw.min()} - {color_raw.max()}")
        # print(f"Original Depth range: {depth_raw.min()} - {depth_raw.max()}")
        
        color_hd = cv2.resize(color_raw, (1920, 1080), interpolation=cv2.INTER_CUBIC)
        depth_hd = cv2.resize(depth_raw, (1920, 1080), interpolation=cv2.INTER_NEAREST)
        
        # print(f"Resized RGB range: {color_hd.min()} - {color_hd.max()}")
        # print(f"Resized Depth range: {depth_hd.min()} - {depth_hd.max()}")
        # Save the resized images
        # resized_rgb_path = os.path.join(resized_rgb_dir, rgb_file)
        # resized_depth_path = os.path.join(resized_depth_dir, depth_file)
        # cv2.imwrite(resized_rgb_path, color_hd)
        # cv2.imwrite(resized_depth_path, depth_hd)

        # Continue processing for point clouds as before
        depth_hd = o3d.geometry.Image(depth_hd)
        color_hd = o3d.geometry.Image(color_hd)

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_hd, depth_hd, convert_rgb_to_intensity=False)
        fx, fy, cx, cy = intrinsics
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(width=1920, height=1080, fx=fx, fy=fy, cx=cx, cy=cy)
        )
        pcd.transform([[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        pointcloud_path = os.path.join(point_dir, rgb_file.replace('.png', '.pcd'))
        o3d.io.write_point_cloud(pointcloud_path, pcd)
        pointcloud_count += 1

    print(f"Total number of point clouds generated for {point_dir}: {pointcloud_count}")

# Process each set of directories
for key, value in directories.items():
    process_images_and_generate_pointclouds(value)


In [14]:
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os
import pyquaternion as pyq

def set_camera_view(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1070.1000, fy=939.8100, cx=960.0, cy=549.6520)
    camera_parameters.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)


def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])


def position_to_nparray(position_str):
    try:
        if isinstance(position_str, str):
            position = ast.literal_eval(position_str)  # Convert string to dictionary
        return np.array([float(position['x']), float(position['y']), float(position['z'])])
    except Exception as e:
        print("Failed to parse position:", position_str)
        raise e

def quaternion_to_nparray(quaternion_str):
    if quaternion_str == 'nan':
        return np.array([1, 0, 0, 0])  # Default quaternion (no rotation)
    quaternion = ast.literal_eval(quaternion_str)
    return pyq.Quaternion(w=float(quaternion['w']), x=float(quaternion['x']), y=float(quaternion['y']), z=float(quaternion['z']))

def read_tracker_data(file_path):
    positions = []
    quaternions = []
    grippers = []
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['left_pos'] == 'nan' or row['left_ori'] == 'nan':
                print(f"Skipping row due to 'nan' values: {row}")
                continue  # Skip this row
            
            position = position_to_nparray(row['left_pos'])
            quaternion = quaternion_to_nparray(row['left_ori'])
            gripper = int(row["left_gripper"])
            positions.append(position)
            quaternions.append(quaternion)
            grippers.append(gripper)
    return positions, quaternions, grippers


def read_tracker_data_change(file_path):
    
    transformed_positions = []
    positions=[]
    transformed_quaternions = []
    quaternions = []
    grippers = []
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['left_pos'] == 'nan' or row['left_ori'] == 'nan':
                print(f"Skipping row due to 'nan' values: {row}")
                continue  # Skip this row
            
            position = position_to_nparray(row['left_pos'])
            quaternion = quaternion_to_nparray(row['left_ori'])
            gripper = int(row["left_gripper"])
            positions.append(position)
            quaternions.append(quaternion)
            grippers.append(gripper)
        for pos in positions:
        # Apply the transformation (-y, z, -x-1.1)
            new_x = -pos[1]  # -y becomes new x
            new_y = pos[2]-1.1  # z becomes new y
            new_z = -pos[0]   # -x-1.1 becomes new z
            transformed_position = np.array([new_x, new_y, new_z])
            transformed_positions.append(transformed_position)
        for quaternion in quaternions:
            # Apply the transformation (z_90, y_90, z_90)

            y_90 = pyq.Quaternion(axis=[0, 1, 0], angle=np.pi/2)
            z_90 = pyq.Quaternion(axis=[0, 0, 1], angle=np.pi/2)
            transformed_quaternion = z_90 * (y_90 * quaternion)  # Apply reverse rotations
            transformed_quaternions.append(transformed_quaternion)
    
    # Return transformed positions along with the unaltered quaternions and grippers
    return transformed_positions, transformed_quaternions, grippers



def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd


In [15]:
tracker1_position, tracker1_quaternion, tracker1_gripper = read_tracker_data("/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_brown45/processed_output_zed_transform.csv")
# tracker4_position, tracker4_quaternion, tracker4_gripper= read_tracker_data("/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/bag/tracker_4/tracker_left_zed_modified.csv")

In [16]:
def quaternion_to_array(quat):
    return np.array([quat.w, quat.x, quat.y, quat.z])

index = 20
# pcd_directory_path_zed_mask = "/media/ubb/4T/human_demonstration/grasp_point_all_sample/lsshirt_bluedot/pointcloud"
# pcd_directory_path_zedtop_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_top"
# pcd_directory_path_zedside_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_side"
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_brown45/pointcloud"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)



vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

translation_zed = [0.139, -0.68, -1.52] # front modified leoa
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo


transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop, quaternion_zedtop_ryd)


vis.add_geometry(transformed_pcd_zed)

colors = {0: [1, 0, 0], 1: [128/255, 0, 128/255]} 
color_tracker1 = colors[tracker1_gripper[index]]


sphere3 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
sphere3.translate(tracker1_position[index]) 
print(tracker1_position[index])
sphere3.paint_uniform_color(color_tracker1) 
vis.add_geometry(sphere3)
frame3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
quat_array = quaternion_to_array(tracker1_quaternion[index])
rotation_matrix3 = o3d.geometry.get_rotation_matrix_from_quaternion(quat_array)
frame3.rotate(rotation_matrix3, center=[0, 0, 0])
frame3.translate(tracker1_position[index])
vis.add_geometry(frame3)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()

[-0.15229061 -0.98165332 -0.96541149]


In [10]:
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os

def set_camera_view(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1070.1000, fy=939.8100, cx=960.0, cy=549.6520)
    camera_parameters.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)





def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

index = 0
# pcd_directory_path_zed_mask = "/media/ubb/4T/human_demonstration/grasp_point_all_sample/lsshirt_bluedot/pointcloud"
# pcd_directory_path_zedtop_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_top"
# pcd_directory_path_zedside_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_side"
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"
pcd_directory_path_zedtop = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"
pcd_directory_path_zedside = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)

pcd_files_top = sorted([file for file in os.listdir(pcd_directory_path_zedtop) if file.endswith('.pcd')])
pcd_file_path_top = os.path.join(pcd_directory_path_zedtop, pcd_files_top[index])  # Index 32 for the 33rd file
pcd_zed_top = o3d.io.read_point_cloud(pcd_file_path_top)

pcd_files_side = sorted([file for file in os.listdir(pcd_directory_path_zedside) if file.endswith('.pcd')])
pcd_file_path_side = os.path.join(pcd_directory_path_zedside, pcd_files_side[index])  # Index 32 for the 33rd file
pcd_zed_side = o3d.io.read_point_cloud(pcd_file_path_side)

# pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
# pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
# pcd_zed = o3d.io.read_point_cloud(pcd_file_path)
# print(pcd_file_path)

vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

translation_zed = [0.139, -0.68, -1.52] # front modified leoa
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
# translation_zedtop = [-0.508, -0.308, -0.84] # top modified leo
# quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo

translation_zedtop_ryd = [-0.471, -0.308, -0.81] # top modified leo
quaternion_zedtop_ryd = [0.655,0.611, 0.305, 0.323] # top modified leo

# translation_zedside = [1.0, -1.030, -0.01]  # side modified leo
# quaternion_zedside =  [0.385, -0.687, -0.588,  0.186]

translation_zedside_ryd = [0.99, -1.03, 0.05]  # side modified leo
quaternion_zedside_ryd =  [0.408, -0.662, -0.602,  0.182]

transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop, quaternion_zedtop_ryd)
transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop_ryd, quaternion_zedtop_ryd)
# transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside, quaternion_zedside)
transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside_ryd, quaternion_zedside_ryd)
# transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)



# sphere3 = o3d.geometry.TriangleMesh.create_sphere(radius=0.02, resolution=20)
# sphere3.translate([-0.2123	-1.2682	-0.8168]) 
# # print(tracker1_position[index])
# # sphere3.paint_uniform_color(color_tracker1) 
# vis.add_geometry(sphere3)
# frame3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
# quat_array = quaternion_to_array([0.6593,0.4722,0.5466,-0.2742])
# rotation_matrix3 = o3d.geometry.get_rotation_matrix_from_quaternion(quat_array)
# frame3.rotate(rotation_matrix3, center=[0, 0, 0])
# frame3.translate(tracker1_position[index])
# vis.add_geometry(frame3)


vis.add_geometry(transformed_pcd_zed)
vis.add_geometry(transformed_pcd_zedtop)
vis.add_geometry(transformed_pcd_zedside)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()

create pointcloud

In [14]:
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os

def set_camera_view(vis):
    control = vis.get_view_control()
    camera_parameters = o3d.camera.PinholeCameraParameters()
    camera_parameters.intrinsic.set_intrinsics(width=1920, height=1080, fx=1070.1000, fy=939.8100, cx=960.0, cy=549.6520)
    camera_parameters.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 5],  # Moves the camera back to ensure a good initial view
        [0, 0, 0, 1]
    ])
    control.convert_from_pinhole_camera_parameters(camera_parameters, allow_arbitrary=True)





def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

index = 0
# pcd_directory_path_zed_mask = "/media/ubb/4T/human_demonstration/grasp_point_all_sample/lsshirt_bluedot/pointcloud"
# pcd_directory_path_zedtop_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_top"
# pcd_directory_path_zedside_mask = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/benchmark_trydata/lsshirt_bluedot/pointcloud_side"
pcd_directory_path_zedtop = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"
pcd_directory_path_zed = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"
pcd_directory_path_zedside = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/pointcloud"

pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
pcd_zed = o3d.io.read_point_cloud(pcd_file_path)

pcd_files_top = sorted([file for file in os.listdir(pcd_directory_path_zedtop) if file.endswith('.pcd')])
pcd_file_path_top = os.path.join(pcd_directory_path_zedtop, pcd_files_top[index])  # Index 32 for the 33rd file
pcd_zed_top = o3d.io.read_point_cloud(pcd_file_path_top)

pcd_files_side = sorted([file for file in os.listdir(pcd_directory_path_zedside) if file.endswith('.pcd')])
pcd_file_path_side = os.path.join(pcd_directory_path_zedside, pcd_files_side[index])  # Index 32 for the 33rd file
pcd_zed_side = o3d.io.read_point_cloud(pcd_file_path_side)

# pcd_files = sorted([file for file in os.listdir(pcd_directory_path_zed) if file.endswith('.pcd')])
# pcd_file_path = os.path.join(pcd_directory_path_zed, pcd_files[index])  # Index 32 for the 33rd file
# pcd_zed = o3d.io.read_point_cloud(pcd_file_path)
# print(pcd_file_path)

vis = o3d.visualization.VisualizerWithEditing()
vis.create_window(width=1920, height=1080)

translation_zed = [0.139, -0.68, -1.52] # front modified leoa
quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
# translation_zedtop = [-0.508, -0.308, -0.84] # top modified leo
# quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo

translation_zedtop_ryd = [-0.471, -0.308, -0.81] # top modified leo
quaternion_zedtop_ryd = [0.655,0.611, 0.305, 0.323] # top modified leo

# translation_zedside = [1.0, -1.030, -0.01]  # side modified leo
# quaternion_zedside =  [0.385, -0.687, -0.588,  0.186]

translation_zedside_ryd = [0.99, -1.03, 0.05]  # side modified leo
quaternion_zedside_ryd =  [0.408, -0.662, -0.602,  0.182]

transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop, quaternion_zedtop_ryd)
transformed_pcd_zedtop = transform_point_cloud(pcd_zed_top, translation_zedtop_ryd, quaternion_zedtop_ryd)
# transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside, quaternion_zedside)
# transformed_pcd_zedside = transform_point_cloud(pcd_zed_side, translation_zedside_ryd, quaternion_zedside_ryd)
# transformed_pcd_zed = transform_point_cloud(pcd_zed, translation_zed, quaternion_zed)

vis.add_geometry(transformed_pcd_zed)
vis.add_geometry(transformed_pcd_zedtop)
vis.add_geometry(transformed_pcd_zedside)


origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
vis.add_geometry(origin)

set_camera_view(vis)
vis.run()
vis.destroy_window()